### Bibliothèques

In [2]:
import trimesh
import pandas as pd
import numpy as np
import tensorflow as tf
import os
from tensorflow.keras.layers import Conv3D, Reshape

### Traitement d'un fichier

In [3]:
def one_file_preprocessing(file_path, resolution=30):
    
    mesh = trimesh.load(file_path, file_type='off')  # Load the 3D mesh from the file using trimesh.
    points, _ = trimesh.sample.sample_surface(mesh, count=10000)  # Sample 10,000 points from the surface of the mesh.
    
    # Avoid division by zero by checking if max == min
    diff = points.max(axis=0) - points.min(axis=0)
    diff[diff == 0] = 1  # Avoid division by zero

    points_normalized = (points - points.min(axis=0)) / diff # Normalize the points to fit within a unit cube [0, 1].
    points_voxels = (points_normalized * (resolution - 1)).astype(int)  # Scale the normalized points to the voxel grid resolution and convert them to integers.
        
    grid = np.zeros((resolution, resolution, resolution), dtype=bool)  # Initialize a 3D voxel grid with the given resolution.
    grid[points_voxels[:, 0], points_voxels[:, 1], points_voxels[:, 2]] = True  # Mark the grid cells corresponding to the sampled points as occupied (True).

    voxel_grid = grid.astype(np.float32)  # Convert the voxel grid to float32 for further processing.
        
    voxel_grid_tensor = tf.convert_to_tensor(voxel_grid, dtype=tf.float32)  # Convert the voxel grid to a TensorFlow tensor.
    voxel_grid_tensor = tf.expand_dims(voxel_grid_tensor, axis=-1)  # Add a channel dimension to the tensor (for Conv3D compatibility).
    
    return voxel_grid_tensor  # Return the processed voxel grid tensor.

### Traitement de tous les fichiers

In [4]:
BASE_FOLDER = r"ModelNet40" 
SUB_FOLDER_TRAIN = r"train"
train_data = []
    
def preprocess_all_files(): 
    voxel_grids = []  # Initialize an empty list to store the processed voxel grids.

    for i, category in enumerate(os.listdir(BASE_FOLDER)):
    
        category_path = os.path.join(BASE_FOLDER, category)
        train_path = os.path.join(category_path, SUB_FOLDER_TRAIN)
        for object in os.listdir(train_path) :
            object_path = os.path.join(train_path, object)
            if os.path.isfile(object_path):
                voxel_grid_tensor = one_file_preprocessing(object_path, resolution=30)  # Preprocess the file and obtain the voxel grid tensor.
                voxel_grids.append(voxel_grid_tensor)  # Append the voxel grid tensor to the list.
                train_data.append((voxel_grid_tensor, i+1))
                print(object_path)
    return train_data  # Return the list of all processed voxel grids with there classes

data = preprocess_all_files()

ModelNet40\airplane\train\airplane_0001.off
ModelNet40\airplane\train\airplane_0002.off
ModelNet40\airplane\train\airplane_0003.off
ModelNet40\airplane\train\airplane_0004.off
ModelNet40\airplane\train\airplane_0005.off
ModelNet40\airplane\train\airplane_0006.off
ModelNet40\airplane\train\airplane_0007.off
ModelNet40\airplane\train\airplane_0008.off
ModelNet40\airplane\train\airplane_0009.off
ModelNet40\airplane\train\airplane_0010.off
ModelNet40\airplane\train\airplane_0011.off
ModelNet40\airplane\train\airplane_0012.off
ModelNet40\airplane\train\airplane_0013.off
ModelNet40\airplane\train\airplane_0014.off
ModelNet40\airplane\train\airplane_0015.off
ModelNet40\airplane\train\airplane_0016.off
ModelNet40\airplane\train\airplane_0017.off
ModelNet40\airplane\train\airplane_0018.off
ModelNet40\airplane\train\airplane_0019.off
ModelNet40\airplane\train\airplane_0020.off
ModelNet40\airplane\train\airplane_0021.off
ModelNet40\airplane\train\airplane_0022.off
ModelNet40\airplane\train\airpla

### Souvegarder la variable

In [5]:
import pickle

# Save the variable to a file
with open('data.pkl', 'wb') as f:
    pickle.dump(data, f)